<a href="https://colab.research.google.com/github/Vish4github/Projects/blob/master/Sentiment_Classification_NB_%7C_Logistic_%7C_NNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#%pip install fastai --upgrade

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai import*
from fastai.text import*
from fastai.vision import *

In [0]:
import sklearn.feature_extraction.text as sklearn_text

In [0]:
#?? URLs

In [6]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/root/.fastai/data/imdb_sample')

In [0]:
df=pd.read_csv(path/'texts.csv')

In [8]:
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [9]:
movie_reviews = (TextList.from_csv(path,'texts.csv',cols='text').split_from_df(col=2).label_from_df(cols=0))  #can help split data into test , validation etc - Here we are getting rid of col2 and saying col 0 is the label column

In [10]:
movie_reviews.valid.x[0],movie_reviews.valid.y[0]

(Text [ 2  5 21 71 ... 15  5  0 52], Category 1)

In [11]:
len(movie_reviews.train.x),len(movie_reviews.valid.x)

(800, 200)

In [12]:
len(movie_reviews.vocab.itos),len(movie_reviews.vocab.stoi)  #itos - integer to string and stoi - string to integer; the reason the nos are different - many words would be mapped as unknown, many as capital etc

(6008, 19159)

In [13]:
movie_reviews.vocab.itos[:10]  #based on frequency - what is itos - integer to strings - the movie reviews are stored as numbers(since the algo uses numbers and not strings) so if we want to see the actual words we need to use itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the']

In [14]:
  c= Counter([4,2,8,8,8,4])  #creates a dictionary with values
  c.values()


dict_values([2, 1, 3])

In [15]:
c.keys()

dict_keys([4, 2, 8])

In [16]:
c

Counter({2: 1, 4: 2, 8: 3})

A matrix with lots of zeroes is called . For sparse matrices a lot of memory can be saved

In [0]:
def get_doc_term_matrix(text_list, n_terms):
    
    # inputs:
    #    text_list, a TextList object
    #    n_terms, the number of tokens in our IMDb vocabulary
    
    # output: 
    #    the CSR format sparse representation of the document-term matrix in the form of a
    #    scipy.sparse.csr.csr_matrix object

    
    # initialize arrays
    values = []
    column_indices = []
    row_pointer = []
    row_pointer.append(0)

    # from the TextList object
    for _, doc in enumerate(text_list):
        feature_counter = Counter(doc.data)
        column_indices.extend(feature_counter.keys())
        values.extend(feature_counter.values())
        # Tack on N (number of nonzero elements in the matrix) to the end of the row_pointer array
        row_pointer.append(len(values))
        
    return scipy.sparse.csr_matrix((values, column_indices, row_pointer),
                                   shape=(len(row_pointer) - 1, n_terms),
                                   dtype=int)

In [45]:
%%time
train_doc_term = get_doc_term_matrix(movie_reviews.train.x, len(movie_reviews.vocab.itos))
val_doc_term = get_doc_term_matrix(movie_reviews.valid.x,len(movie_reviews.vocab.itos))

CPU times: user 171 ms, sys: 1.21 ms, total: 172 ms
Wall time: 173 ms


NAIVE BAYES

In [20]:
movie_reviews.y.classes

['negative', 'positive']

In [0]:
x = train_doc_term
y = movie_reviews.train.y
val_y = movie_reviews.valid.y
v = movie_reviews.vocab

In [0]:
positive = y.c2i['positive']
negative =y.c2i['negative']

In [50]:
x.shape

(800, 6008)

In [0]:
import numpy as np
#to calculate the log count ratios

p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))   
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

In [53]:
p0[v.stoi['loved']],p1[v.stoi['loved']]  # no of times word loved appeared in negative and positive reviews

(12, 29)

In [54]:
p0[v.stoi['hate']],p1[v.stoi['hate']] 

(13, 5)

In [69]:
v.stoi['hated']  # the integer value of word hate 

1977

In [74]:
a = np.argwhere((x[:,1977]>0))[:,0];a    #rows or reviews from the dtm where the count of word hate > 1

array([ 15,  49, 304, 351, 393, 612, 695, 773], dtype=int32)

In [75]:
b = np.argwhere(y.items==positive)[:,0];b    #rows corresponding to positive reviews

array([  1,   3,  10,  11, ..., 787, 789, 790, 797])

In [76]:
set(a).intersection(set(b))  # rows where the review is positive but it has the word hate

{393, 612, 695}

In [77]:
review = movie_reviews.train.x[393];
print(''.join(review.text))

xxbos xxmaj there are numerous films relating to xxup xxunk , but xxmaj mother xxmaj night is quite xxunk among them : xxmaj in this film , we are introduced to xxmaj howard xxmaj campbell ( xxmaj nolte ) , an xxmaj american living in xxmaj berlin and married to a xxmaj german , xxmaj xxunk xxmaj xxunk ( xxmaj lee ) , who decides to accept the role of a spy : xxmaj more specifically , a xxup cia agent xxmaj major xxmaj xxunk ( xxmaj goodman ) recruits xxmaj campbell who becomes a xxmaj nazi xxunk in order to enter the highest xxunk of the xxmaj hitler xxunk . xxmaj however , the deal is that the xxup us xxmaj government will never xxunk xxmaj campbell 's role in the war for national security reasons , and so xxmaj campbell becomes a hated figure across the xxup us . xxmaj after the war , he tries to xxunk his identity , but the past comes back and xxunk him . xxmaj his only " friend " is xxmaj xxunk , but even he can not do much for the xxunk of events that fall upon poor xxmaj campbel

Applying Naive Bayes

In [80]:
pr1 = (p1+1)/((y.items==positive).sum()+1)  #adding 1 in num and deno for numerical stability
pr0 = (p0+1)/((y.items==negative).sum()+1)

r = np.log(pr1/pr0);r

array([-0.015348,  0.084839,  0.      ,  0.084839, ...,  1.471133, -1.301455, -1.301455, -1.301455])

Vocab most likely associated with postive and negative reviews

In [0]:
biggest = np.argpartition(r,-10)[-10:]
smallest = np.argpartition(r,10)[:10]

In [83]:
#most positive
[v.itos[k] for k in biggest]

['han',
 'jabba',
 'gilliam',
 'davies',
 'noir',
 'felix',
 'jimmy',
 'astaire',
 'fanfan',
 'biko']

In [84]:
np.argmax(train_doc_term[:,v.stoi['jimmy']])

741

In [86]:
movie_reviews.train.x[741].text  #jimmy appears multiple times in this positive review

'xxbos * * * xxup spoilers * * * xxmaj all too , in real life as well as in the movies , familiar story that happens to many young men who are put in a war zone with a gun , or xxunk , in their hands . xxmaj the case of young and innocent , in never xxunk or firing a gun , xxmaj jimmy xxmaj davis , xxmaj xxunk xxmaj tone , has been repeated thousands of times over the xxunk when men , like xxmaj jimmy xxmaj davis , are forced to take up arms for their country . \n \n  xxmaj jimmy who at first wanted to be kicked out of the xxup us xxmaj army but was encouraged to stay , by being xxunk in the mouth , by his good friend xxmaj fred xxup xxunk xxmaj willis , xxmaj xxunk xxmaj tracy , ended up on the front lines in xxmaj france . xxmaj with xxmaj jimmy \'s unit xxunk down by a xxmaj german machine gun nest he single xxunk put it out of xxunk picking off some half dozen xxmaj german soldiers from the xxunk of a nearby church xxunk . xxmaj it was when xxmaj jimmy xxunk down the last surviving

In [0]:
b = np.log((y.items==positive).mean()/(y.items==negative).mean())  #ratio of avg postive reviews to neg reviews

In [0]:
preds = (val_doc_term @ r + b) > 0 

In [90]:
(preds == val_y.items).mean()

0.645

Binarized Naive Bayes - maybe it only matters to know whether a word is in the review or not and not the frequency 

In [0]:
x = train_doc_term.sign()
y= movie_reviews.train.y

In [0]:
negative = y.c2i['negative']
positive = y.c2i['positive']

In [0]:
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

In [0]:
pr1 = (p1+1)/((y.items==positive).sum()+1)
pr0 = (p0+1)/((y.items==negative).sum()+1)

In [0]:
r = np.log(pr1/pr0)
b = np.log((y.items==positive).mean()/(y.items==negative).mean())
preds = (val_doc_term @ r +b) >0

In [97]:
(preds == val_y.items).mean()

0.66

Logistic Regression